In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np 
import re 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer 
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report , confusion_matrix

In [2]:
train = pd.read_csv('drugsComTrain_raw.csv')
#train = df.iloc[: , :-1]
train['label'] = '11'
test = pd.read_csv('drugsComTest_raw.csv')
test['label'] = '22'

In [3]:
combined = train.append(test)

In [4]:
train.shape, test.shape ,  combined.shape

((161297, 8), (53766, 8), (215063, 8))

In [5]:
combined.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount,label
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27,11
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192,11
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,11
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10,11
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37,11


### convert the rating column from numeric one to categorical [0 1 2 ] negative , naurtal , positive 

In [6]:
def to_cat (value):
    if (value <= 4 ):
        val = 0
    elif (value < 8 ):
        val = 1
    else:
        val = 2
    return val

In [7]:
combined['rating'] = combined['rating'].apply(to_cat)

In [8]:
combined['rating'].value_counts()

2    129759
0     53572
1     31732
Name: rating, dtype: int64

In [9]:
x  = combined['review']
y  = combined['rating']

In [10]:
combined.isna().sum()

uniqueID          0
drugName          0
condition      1194
review            0
rating            0
date              0
usefulCount       0
label             0
dtype: int64

### apply the cleaning process to the review column to use it with NLP to make classification 

In [13]:
import nltk 
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\A7md\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
lemmatizer = WordNetLemmatizer()

def clean_text (review):
    review =re.sub('[^a-zA-Z]' ,' ', review)
    review = review.lower()
    review = review.split(' ')
    review = [lemmatizer.lemmatize(word) for word in review if not word in set (stopwords.words('english'))]
    review = ' '.join(review)
    return review

In [12]:
x.head()

0    "It has no side effect, I take it in combinati...
1    "My son is halfway through his fourth week of ...
2    "I used to take another oral contraceptive, wh...
3    "This is my first time using any form of birth...
4    "Suboxone has completely turned my life around...
Name: review, dtype: object

In [98]:
#x_tt = x[:3000]
#y_tt = y[:3000]
#x_tt.shape

(3000,)

In [14]:
# apply the function to the the review column and save it to external csv file , because it take much more time  
x_new = x.apply(clean_text)

In [15]:
x_new.to_csv('x_cleansed.csv')
#data.dropna(inplace=True)

In [16]:
x_new.head()

0     side effect  take combination bystolic   mg f...
1     son halfway fourth week intuniv  became conce...
2     used take another oral contraceptive     pill...
3     first time using form birth control       gla...
4     suboxone completely turned life around   feel...
Name: review, dtype: object

In [17]:
tfidf = TfidfVectorizer()
# x = tfidf.fit_transform(data["title"]).toarray()

In [18]:
x_train,x_test,y_train,y_test = train_test_split(x_new,y,test_size=0.25)

In [19]:
print (x_train.shape, y_train.shape)
print (x_test.shape, y_test.shape)

(161297,) (161297,)
(53766,) (53766,)


In [20]:
model = tfidf.fit_transform(x_train)
predection = tfidf.transform(x_test)

In [23]:
lr = LogisticRegression(random_state=0)
lr.fit(model, y_train)
print (lr.score(predection,y_test))
y_pred = lr.predict(predection)
print(classification_report(y_test, y_pred))


C:\Users\A7md\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\A7md\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.751329836699773
              precision    recall  f1-score   support

           0       0.70      0.70      0.70     13277
           1       0.50      0.12      0.20      7960
           2       0.78      0.93      0.85     32529

   micro avg       0.75      0.75      0.75     53766
   macro avg       0.66      0.58      0.58     53766
weighted avg       0.72      0.75      0.71     53766



In [22]:
confusion_matrix(y_test, y_pred)

array([[ 9304,   415,  3558],
       [ 2112,   969,  4879],
       [ 1858,   548, 30123]], dtype=int64)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(random_state=0,n_estimators=5)
regr.fit(model, y_train)
regr.score(predection,y_test)
y_pred = regr.predict(predection)
print(classification_report(y_test, y_pred))